In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sercandikici/merged-dataset-electricty-weather-for-modelling
! unzip merged-dataset-electricty-weather-for-modelling.zip

In [ ]:
from prophet import Prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

def root_mean_squared_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def r_squared(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_y_true = np.mean(y_true)
    ss_total = np.sum((y_true - mean_y_true) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    return r2


In [ ]:
df_merged = pd.read_csv("merged_data.csv")
df_merged['settlement_date'] = pd.to_datetime(df_merged['settlement_date'])
df_merged["tsd"] = np.square(df_merged["tsd"])
df_merged

In [ ]:
split_from = 90*12 #train test split is from 90days
freq = "2H"
days_to_predict=7
periods = days_to_predict*12
mcmc_samples = 30

In [ ]:
df_model_3 = df_merged[["tsd","settlement_date","temp"]]
df_model_3.columns=["y","ds","temp"]
df_model_3

In [ ]:
train_data_3 = df_model_3[:-split_from]
test_data_3 = df_model_3[-split_from:]

In [ ]:
# Define the parameter grid
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # Store the MAPEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    # Initialize Prophet model with given params
    m = Prophet(**params,mcmc_samples=mcmc_samples)
    m.add_country_holidays(country_name="UK")
    m.add_regressor("temp", mode="additive")

    # Fit the model
    m.fit(train_data_3)

    # Make future dataframe
    future = m.make_future_dataframe(periods=periods, freq=freq)

    # Add regressors to future dataframe
    train_idx = future["ds"].isin(train_data_3["ds"])
    test_idx = ~train_idx
    reg = ["temp"]
    for r in reg:
        future.loc[train_idx, r] = train_data_3[r].to_list()
        future.loc[test_idx, r] = test_data_3.iloc[:periods][r].to_list()

    # Predict and calculate MAPE
    forecast = m.predict(future)
    forecast_days = forecast[forecast["ds"] >= test_data_3["ds"].iloc[0]]
    test_days = test_data_3[(test_data_3["ds"] >= test_data_3["ds"].iloc[0]) & (test_data_3["ds"] <= forecast_days["ds"].iloc[-1])]
    mape = mean_absolute_percentage_error(test_days["y"], forecast_days["yhat"])
    mapes.append(mape)

# Combine parameters and corresponding MAPEs into a DataFrame
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
print(tuning_results)
